<a href="https://colab.research.google.com/github/amanzour/ViterbiRecursiveSQL/blob/main/Recursive_SQL_Viterbi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to implement **Recursive SQL**: An example of the **Viterbi Algorithm**

# 1.&nbsp;Abstract

The **Viterbi algorithm** obtains the most likely sequence of hidden states from amongst a series of possibilities interlinked with hidden states.      
[Wikipedia](https://en.wikipedia.org/wiki/Viterbi_algorithm). On the other hand, SQL has been widely used for fast reading and processing of data. Here, we will present an implementation of the Viterbi algorithm using the recursive common table expression (CTE) feature of SQL.      
 
The Viterbi algorithm is mainly implemented through dynamic programming. Here, it is shown that the algorithm can also be implemented via a recursive Common Table Expression (CTE) in SQL.    
 
First, a simple example of the viterbi algorithm is implemented in  Python. Then, the SQL equivalent of that example is presented.     


# 2.&nbsp;Python implementation

The code `viterbi.py` is the Python implementation. Notations closely follow that of wikipedia. In this example, we have 6 observations and two hidden states at each observation.

In [ ]:
import numpy as np
 
# Viterbi Algorithm Implementation
 
# observation space
O = np.array([0, 1, 2], dtype=int)
 
# state space
S = np.array([0, 1], dtype=int)
 
# initial probabilities
PI = np.array([0.5, 0.5])
 
# observations
Y = np.array([2, 2, 2, 0, 2, 1], dtype=int)
 
# transition matrix
A = np.array([[0.75, 0.25],[0.4, 0.6]])
 
# emission matrix
B = np.array([[0.33, 0.54, 0.1],[0.5, 0.25, 0.25]])
 
 
# probabilitiy of the most likely path up to each observation
T_1 = np.zeros((len(S),len(Y)), dtype=float)
 
# states of the most likely path up to each observation
T_2 = np.zeros((len(S),len(Y)), dtype=int)
 
# initialization of state probabilities given the first observation
for i in range(len(S)):
   T_1[i,0] = PI[i]*B[i,Y[0]]
   T_2[i,0] = -1
 
# iterating over observations and states and storing only the most optimum paths up to the observation
for j in range(1,len(Y)):
   for i in range(len(S)):
       curval=0
       curk = -1
       for k in range(len(S)):  
           if T_1[k,j-1]*A[k,i]*B[i,Y[j]] > curval:
               curval = T_1[k,j-1]*A[k,i]*B[i,Y[j]]
               curk = k
       T_1[i,j] = curval
       T_2[i,j] = curk
 
 
# the optimum path
X = np.zeros(len(Y), dtype=int)
X[-1] = np.argmax(T_1[:,-1])
 
for j in range(len(Y)-1,0,-1):
   X[j-1] = T_2[X[j],j]
 
print("X =" + str(X))


Hidden path with highest likelihood would contain states `X =[1 1 1 1 1 0]`, where 0 and 1 are first and second states, respectively. The corresponding likelihood for this set of states is stored in `T[i=6,j=0]` and is `2.7337500e-05`.

# 3.&nbsp;SQL implementation

The SQL implementation uses the same notations. It constructs tables: PI, A, B, and Y. Here we are using identical values to obtain identical deterministic results. For a more comprehensive description of implementing Viterbia via recursive SQL, please see “Probabilistic Declarative Information Extraction" by Wang et. al. First, let's initialize our tables:

```sql
--PostgreSQL 14

--Initialization
drop table if exists PI;
create table PI(s_init int NOT NULL UNIQUE, w float);
insert into PI values(1, 0.5); 
insert into PI values(2, 0.5);
 
drop table if exists A;
create table A(s_source int, s_dest int, w float);
insert into A values(1, 1, 0.75);
insert into A values(1, 2, 0.25);
insert into A values(2, 1, 0.4);
insert into A values(2, 2, 0.6);
 
drop table if exists B;
create table B(s int, y int, prob float);
insert into B values(1, 1, 0.33);
insert into B values(1, 2, 0.54);
insert into B values(1, 3, 0.1);
insert into B values(2, 1, 0.5);
insert into B values(2, 2, 0.25);
insert into B values(2, 3, 0.25);
 
drop table if exists Y;
create table Y(j SERIAL, y int);
insert into Y (y) values(3);
insert into Y (y) values(3);
insert into Y (y) values(3);
insert into Y (y) values(1);
insert into Y (y) values(3);
insert into Y (y) values(2);
```

The `viterbi` CTE recursively keeps the most likely paths at each state. 
The columns of the `viterbi` table are `j` (current observation state), `s` (current hidden state), `x` (optimum path to the current hidden state), `likelihood` (likelihood of being in the given current hidden state
 
First, let’s fill the table for our first observed `Y`.


```sql
select Y.j, PI.s_init, array[0] as x, PI.w*B.prob as likelihood from PI 
    inner join B on PI.s_init=B.s 
    inner join Y on B.y=Y.y and Y.j = 1 
```

Here, `array[0]` is meant to contain a dummy previous hidden state, since we are already at the first hidden state. The rest of the columns are the result of linking the initial probabilities table to the corresponding states on the emission table to get the probability of the first observed `Y` (last inner join) while being in that hidden state.
 
In order to explain the recursive part of the query, let’s not do the maximization at this point and keep all possible paths leading to the 6th observation `Y`. Here, let’s add another column `s_prev` just to keep previous states as well as current states. I am listing them in the table below to show that this is just an intermediate variable as not really needed in the final code. I will also limit the recursion to only one step by adding `and Y.j <= 2` to the last line:

| item | in this line |
| --- | --- |
| `s_prev` | `with recursive viterbi (j, s_prev, s, x, likelihood) as` |
| `0` | `select Y.j, 0, PI.s_init, ...` |
| `and Y.j <= 2` | `inner join Y on B.y=Y.y and viterbi.j + 1 = Y.j and Y.j <= 2` |
| `s_prev` | `select j, s_prev, s, x \|\| array[s] as states, likelihood from viterbi` |


```sql
with recursive viterbi (j, s_prev, s, x, likelihood) as
(
select Y.j, 0, PI.s_init, array[0] as x, PI.w*B.prob as likelihood from PI 
    inner join B on PI.s_init=B.s 
    inner join Y on B.y=Y.y and Y.j = 1 
union all  
select Y.j, A.s_source, A.s_dest, x || array[A.s_source] as x, viterbi.likelihood*A.w*B.prob as likelihood from A
    inner join viterbi on A.s_source = viterbi.s
    inner join B on A.s_dest=B.s 
    inner join Y on B.y=Y.y and viterbi.j + 1 = Y.j and Y.j <= 2
)
select j, s_prev, s, x || array[s] as states, likelihood from viterbi
```

Here, in the recursive component, shown in green, we are linking records in our original table `viterbi` with those rows of the transition matrix `A` whose previous state is the same as the states of the `viterbi` table. Other tables are added in a similar fashion to the initial table. There is one extra point. Since we are considering observations, one at a time, we are only adding the observation, whose instance is one more than the current instance, hence condition `viterbi.j + 1 = Y.j`.
Running the above code results in the following table:


| j | s_prev | s | states | likelihood
| --- | --- | --- | --- | --- |
| <font color='gray'>1</font> | <font color='gray'>0 </font>| <font color='gray'>1</font> | <font color='gray'>{0,1}</font> | <font color='gray'>0.05</font> |
| <font color='gray'>1</font> | <font color='gray'>0</font> | <font color='gray'>2</font> | <font color='gray'>{0,2}</font> | <font color='gray'>0.125</font> |
| 2 | 1 | 1 | {0,1,1} | 0.00375 |
| <font color='red'>2</font> | <font color='red'>2</font> | <font color='red'>1</font> | <font color='red'>{0,2,1}</font> | <font color='red'>0.005</font> |
| 2 | 1 | 2 | {0,1,2} | 0.003125 |
| <font color='red'>2</font> | <font color='red'>2</font> | <font color='red'>2</font> | <font color='red'>{0,2,2}</font> | <font color='red'>0.01875</font> |

Recorded corresponding to the non-recursive part of the query are grayed out. As you can see, all paths corresponding to up to j=2 are recorded. Omitting expression <font color='red'>`and Y.j <= 2`</font> will result in the computation of all paths, where at each step of the recursion, only the current states (viterbi table) are considered in linking with the next state. In the above situation, the number of records will grow exponentially, as we are simply recording all possible paths. In the Viterbi algorithm, given each state, it suffices to keep only those paths that have the highest likelihood ending in that state. In the above table, these paths are colored red. We can simply discard the two black records. This can be achieved by nesting this table in another one, where we only choose those records that have maximum likelihood (that rank first) in each group of records having the same state (partitioned by `s`).

# 4.&nbsp;Command

```sql
rank() over (partition by s_source order by likelihood desc)
```

Will do this trick. Nesting the resulting table in yet another table, only to eliminate the extra columns will result in the final query. Note that the nesting part seems to lead to errors in mysql (at least the version I used). In postgreSQL, however, this is implemented smoothly.

After the recursion, the very last step becomes tricky. Here, we have to explicitly find the maximum path and append its corresponding state to the optimum path `x` and also have to get rid of the first element in array of steps, since it was a dummy variable to start with (see this `x[2:cardinality(x)] || array[s] as states` in the third line from bottum).

The output of the following SQL code is identical to the Python code above, except that firest and second states are identified as 1 and 2 in SQL, whereas they are shown as 0 and 1 in Pyton, sepectively.

```sql
-- optimization
with recursive viterbi (j, s, x, likelihood) as
(
select Y.j, PI.s_init, array[0] as x, PI.w*B.prob as likelihood from PI 
    inner join B on PI.s_init=B.s 
    inner join Y on B.y=Y.y and Y.j = 1 
union all
 -- keeping only the columns we would need (read this line last)
select j, s_dest, x, likelihood from 
 -- keeping only rows with maximum likelihoods (read this line second)
(select j, s_dest, x, likelihood, rank() over (partition by s_source order by likelihood desc) as ranking from 
 -- the actual recursive all (read this block first)
 (select Y.j, A.s_source, A.s_dest, x || array[A.s_source] as x, viterbi.likelihood*A.w*B.prob as likelihood from A
    inner join viterbi on A.s_source = viterbi.s
    inner join B on A.s_dest=B.s 
    inner join Y on B.y=Y.y and viterbi.j + 1 = Y.j
    ) as t) as d where d.ranking = 1
)
select j, s, x[2:cardinality(x)] || array[s] as states, likelihood from viterbi
where j in (select max(j) from viterbi)
order by likelihood desc limit 1;
```